In [ ]:
import pandas as pd

In [ ]:
%run ../modules/utils.ipynb
%run ../modules/cds.ipynb
%run ../modules/preprocessing.ipynb

In [ ]:
path = '/Users/xyuan/Documents/ChenLab/10Jun2021/degraded/23_t_Cys.xlsx'
df_wild = load_data(path)
path = '/Users/xyuan/Documents/ChenLab/10Jun2021/degraded/24_t_Cys_A.xlsx'
df_alkb = load_data(path)
df_wild.shape, df_alkb.shape

In [ ]:
df_wild = df_wild[df_wild.Mass>1500]
df_alkb = df_alkb[df_alkb.Mass>1500]
df_wild.shape, df_alkb.shape

In [ ]:
_, df_alkb_only = diff_dfs(df_wild, df_alkb)
df_alkb_only.shape

In [ ]:
plotly_zone(df_alkb_only)

In [ ]:
import multiprocessing
from collections import namedtuple
ShiftHit = namedtuple('ShiftHit', 'shift hit')

def func(df_s1, df_s2, shift):
    dfm = match_dfs(df_s1, df_s2, shift=shift)
    sh = ShiftHit(shift=shift, hit=dfm.shape[0])
    return sh

PROCESSES = 8
# params = [(df_intact_wild2, df_intact_alkb2_only, shift) for shift in np.arange(-64, 64, 1)]
params = [(df_wild, df_alkb_only, shift) for shift in np.arange(-64, 64, 1)]
# params = [(df_comm_wild2, df_comm_wild2, shift) for shift in np.arange(1, 64, 1)]
# params = [(df_wild2, df_alkb2, shift) for shift in np.arange(1, 100, 1)]
# params = [(df_sample, shift) for shift in np.arange(1, 64, 0.1)]
with multiprocessing.Pool(PROCESSES) as pool:
    shift_hits = pool.starmap(func, params)

df_shift_hits = pd.DataFrame(shift_hits)    

In [ ]:
fig = px.scatter(df_shift_hits, x='shift', y='hit', labels={'shift': 'Mass Shift', 'hit': 'Hit Count'})
fig.show()

In [ ]:
df_shift_hits.sort_values('hit', ascending=False).reset_index(drop=True).iloc[:10].copy()